In [1]:
import pandas as pd
from Bio import Medline
from Bio import Entrez
import string

Entrez.email = "1219131962@qq.com"
Entrez.api_key = "6fcb01c2989c35aa1da3d9d0118abedfc409"
PMID_info = []
CGM_result_dict = {}


def get_keywords(PMID_list):
    """Thank to Dr. Mao Zhitao for his help in this function."""
    
    for i in PMID_list:
        # print(i)
        handle = Entrez.efetch(db="pubmed", id=i, rettype="medline", retmode="text")
        records = Medline.parse(handle)
        records = list(records)

        for record in records:
            title = record.get("TI", "?")
            abstract = record.get("AB", "?")
            keywords = record.get("OT", "?")
            Journal = record.get("JT", "?")
            country = record.get("AD", "?")[0].split(", ")[-1].strip(string.punctuation)
            pmid = record.get("PMID", "?")
            Author = record.get("FAU", "?")
            doi = record.get("SO", "?")
            date = record.get("DEP", "?")
            year = record.get("DP", "?")
            institution = record.get("AD", "?")[0]

            PMID_info.append(
                {
                    "PMID": pmid,
                    "DOI": doi,
                    "Author": Author,
                    "Title": title,
                    "Abstract": abstract,
                    "Keywords": keywords,
                    "Journal": Journal,
                    "Institution": institution,
                    "Country": country,
                    "Date": date,
                    "Year": year,
                }
            )

    for key, value in CGM_result_dict.items():
        PMID_info.append([key, value])

    return PMID_info

先去pubmed上根据菌种的名字，检索下载所有发表的csv文件

In [2]:
"""
测试文献解析代码
"""
df = pd.read_csv('pmid-demo.txt', sep='\t', header=None, names=['PMID'])

# 删除重复值
df = df.dropna(subset=['PMID'])
df = df.reset_index(drop=True)

get_keywords(list(df.iloc[:, 0]))
df_info = pd.DataFrame(PMID_info)

df_info['DOI'] = df_info['DOI'].str.extract(r'(10\.\d{4,9}\/[-._;()/:A-Za-z0-9]+)')
df_info['DOI'] = df_info['DOI'].str.rstrip('.')

In [3]:
df_info

,PMID,DOI,Author,Title,Abstract,Keywords,Journal,Institution,Country,Date,Year
0,25025273,10.3109/07388551.2014.923985,"[Singh, Bijender]",Myceliophthora thermophila syn. Sporotrichum t...,Myceliophthora thermophila syn. Sporotrichum t...,"[Biomolecules, Myceliophthora thermophila, Spo...",Critical reviews in biotechnology,"a Laboratory of Bioprocess Technology, Departm...",India,20140715,2016
1,32640074,10.1111/jam.14774,"[Dos Santos Gomes, A C, Casciatori, F P, Gomes...",Growth kinetics of Myceliophthora thermophila ...,AIMS: This work aimed to estimate the growth o...,"[N-acetylglucosamine, enzymes, growth kinetics...",Journal of applied microbiology,"Instituto de Biociencias, Letras e Ciencias Ex...",Brazil,20200729,2021 Jan


In [4]:
df_Mt1 = pd.read_csv("pubmed/csv-Myceliopht-set.csv")
df_Mt1

,PMID,Title,Authors,Citation,First Author,Journal/Book,Publication Year,Create Date,PMCID,NIHMS ID,DOI
0,25025273,Myceliophthora thermophila syn. Sporotrichum t...,Singh B.,Crit Rev Biotechnol. 2016;36(1):59-69. doi: 10...,Singh B,Crit Rev Biotechnol,2016,2014/07/16,NaN,NaN,10.3109/07388551.2014.923985
1,32640074,Growth kinetics of Myceliophthora thermophila ...,"Dos Santos Gomes AC, Casciatori FP, Gomes E, d...",J Appl Microbiol. 2021 Jan;130(1):90-99. doi: ...,Dos Santos Gomes AC,J Appl Microbiol,2021,2020/07/09,NaN,NaN,10.1111/jam.14774
2,31534479,Myceliophthora thermophila Xyr1 is predominant...,"Dos Santos Gomes AC, Falkoski D, Battaglia E, ...",Biotechnol Biofuels. 2019 Sep 16;12:220. doi: ...,Dos Santos Gomes AC,Biotechnol Biofuels,2019,2019/09/20,PMC6745793,NaN,10.1186/s13068-019-1556-y
3,24995002,Genomic insights into the fungal lignocellulol...,"Karnaouri A, Topakas E, Antonopoulou I, Christ...",Front Microbiol. 2014 Jun 18;5:281. doi: 10.33...,Karnaouri A,Front Microbiol,2014,2014/07/05,PMC4061905,NaN,10.3389/fmicb.2014.00281
4,35450635,Extending the diversity of Myceliophthora ther...,"Sun P, de Munnik M, van Berkel WJH, Kabel MA.",Carbohydr Polym. 2022 Jul 15;288:119373. doi: ...,Sun P,Carbohydr Polym,2022,2022/04/22,NaN,NaN,10.1016/j.carbpol.2022.119373
...,...,...,...,...,...,...,...,...,...,...,...
273,38029111,Highly efficient synergistic activity of an α-...,"Wen J, Miao T, Basit A, Li Q, Tan S, Chen S, A...",Front Microbiol. 2023 Nov 3;14:1230738. doi: 1...,Wen J,Front Microbiol,2023,2023/11/29,PMC10655120,NaN,10.3389/fmicb.2023.1230738
274,33807631,Prospection of Fungal Lignocellulolytic Enzyme...,"Contato AG, de Oliveira TB, Aranha GM, de Frei...",Microorganisms. 2021 Mar 5;9(3):533. doi: 10.3...,Contato AG,Microorganisms,2021,2021/04/03,PMC8000596,NaN,10.3390/microorganisms9030533
275,12817956,Spectroscopic characterization of the Leu513Hi...,"Palmer AE, Szilagyi RK, Cherry JR, Jones A, Xu...",Inorg Chem. 2003 Jun 30;42(13):4006-17. doi: 1...,Palmer AE,Inorg Chem,2003,2003/06/24,NaN,NaN,10.1021/ic026099n
276,30383933,[CHARACTERISTICS OF FUNGAL STRAINS PRODUCING T...,"Krestyanova IN, Sakhibgaraeva LF, Berezina OV,...",Mol Gen Mikrobiol Virusol. 2016;34(3):109-114.,Krestyanova IN,Mol Gen Mikrobiol Virusol,2016,2018/11/02,NaN,NaN,NaN


In [6]:
df_Mt2 = pd.read_csv("pubmed/csv-Thermothel-set.csv")
df_Mt2

,PMID,Title,Authors,Citation,First Author,Journal/Book,Publication Year,Create Date,PMCID,NIHMS ID,DOI
0,35205873,Engineered Fungus Thermothelomyces thermophilu...,"Balabanova L, Seitkalieva A, Yugay Y, Rusapeto...",J Fungi (Basel). 2022 Jan 26;8(2):119. doi: 10...,Balabanova L,J Fungi (Basel),2022,2022/02/25,PMC8877005,NaN,10.3390/jof8020119
1,33461124,Fed-batch production of Thermothelomyces therm...,"Liu E, Segato F, Wilkins MR.",Bioresour Technol. 2021 Apr;325:124700. doi: 1...,Liu E,Bioresour Technol,2021,2021/01/18,NaN,NaN,10.1016/j.biortech.2021.124700
2,37354337,Trichoderma longibrachiatum and thermothelomyc...,"Contato AG, Nogueira KMV, Buckeridge MS, Silva...",Biotechnol Lett. 2023 Sep;45(9):1093-1102. doi...,Contato AG,Biotechnol Lett,2023,2023/06/24,NaN,NaN,10.1007/s10529-023-03395-7
3,31309856,Isolation and characterization of a new cytoto...,"Gao YL, Zhang ML, Wang X, Zhang HD, Huang JZ, ...",Nat Prod Res. 2021 Jun;35(11):1792-1798. doi: ...,Gao YL,Nat Prod Res,2021,2019/07/17,NaN,NaN,10.1080/14786419.2019.1641810
4,32653619,Enzymatic versatility and thermostability of a...,"Kadowaki MAS, Higasi PMR, de Godoy MO, de Araú...",Biochim Biophys Acta Gen Subj. 2020 Oct;1864(1...,Kadowaki MAS,Biochim Biophys Acta Gen Subj,2020,2020/07/13,NaN,NaN,10.1016/j.bbagen.2020.129681
5,36506193,Identification of Thermostable Xylose Reductas...,"Ali N, Aiman A, Shamsi A, Hassan I, Shahid M, ...",ACS Omega. 2022 Nov 17;7(48):44241-44250. doi:...,Ali N,ACS Omega,2022,2022/12/12,PMC9730754,NaN,10.1021/acsomega.2c05690
6,37451383,Staphylococcus aureus microbial biofilms degra...,"Samaniego LVB, Higasi PMR, de Mello Capetti CC...",Int J Biol Macromol. 2023 Aug 30;247:125822. d...,Samaniego LVB,Int J Biol Macromol,2023,2023/07/14,NaN,NaN,10.1016/j.ijbiomac.2023.125822
7,33301505,Safety evaluation of a β-mannanase enzyme prep...,"Kern A, Shanahan D, Buesen R, Geiger D.",PLoS One. 2020 Dec 10;15(12):e0243647. doi: 10...,Kern A,PLoS One,2020,2020/12/10,PMC7728267,NaN,10.1371/journal.pone.0243647
8,33712158,Light-stimulated T. thermophilus two-domain LP...,"Higasi PMR, Velasco JA, Pellegrini VOA, de Ara...",Carbohydr Polym. 2021 May 15;260:117814. doi: ...,Higasi PMR,Carbohydr Polym,2021,2021/03/13,NaN,NaN,10.1016/j.carbpol.2021.117814
9,36866191,Safety and efficacy of a feed additive consist...,EFSA Panel on Additives and Products or Substa...,EFSA J. 2023 Feb 28;21(2):e07873. doi: 10.2903...,EFSA Panel on Additives and Products or Substa...,EFSA J,2023,2023/03/03,PMC9972018,NaN,10.2903/j.efsa.2023.7873


In [8]:
"""
嗜热毁丝霉有两个常用名称，先聚合，然后去重
- 结果显示，Mt这个名字的csv，包含了Tt的.....
"""

# 合并两个dataframe，如果PMID有重复的，则保留第一个
df_Mt = pd.concat([df_Mt1, df_Mt2], ignore_index=True)
df_Mt = df_Mt.drop_duplicates(subset=["PMID"], keep="first")

# 取df_Mt中的PMID和DOI列
df_Mt_filter = df_Mt[["PMID", "DOI", "Publication Year"]]

# 输出DOI为空的PMID
# df_Mt_filter[df_Mt_filter['DOI'].isnull()]

# 将DOI的缺失值填充为空
# df_Mt_filter.loc[:, 'DOI'] = df_Mt_filter['DOI'].fillna('')
df_Mt_filter = df_Mt_filter.copy()
df_Mt_filter["DOI"] = df_Mt_filter["DOI"].fillna("")

# 删除PMID列中的重复值
df_Mt_filter = df_Mt_filter.drop_duplicates(subset=["PMID"], keep="first")

In [9]:
df_Mt_filter

,PMID,DOI,Publication Year
0,25025273,10.3109/07388551.2014.923985,2016
1,32640074,10.1111/jam.14774,2021
2,31534479,10.1186/s13068-019-1556-y,2019
3,24995002,10.3389/fmicb.2014.00281,2014
4,35450635,10.1016/j.carbpol.2022.119373,2022
...,...,...,...
273,38029111,10.3389/fmicb.2023.1230738,2023
274,33807631,10.3390/microorganisms9030533,2021
275,12817956,10.1021/ic026099n,2003
276,30383933,,2016


In [10]:
"""
get_keywords这个函数一般一次最好处理100个PMID，否则容易被封IP
"""

# 将PMID转换为list
PMID_list = list(df_Mt_filter["PMID"])

# 输出PMID的数量
len(PMID_list)

278

In [11]:
# 将PMID分为多个list，每个list最多100个PMID
PMID_list_1 = PMID_list[:100]
PMID_list_2 = PMID_list[100:200]
PMID_list_3 = PMID_list[200:300]

In [12]:
df_list_1 = pd.DataFrame(get_keywords(PMID_list_1))

In [13]:
df_list_2 = pd.DataFrame(get_keywords(PMID_list_2))

In [14]:
df_list_3 = pd.DataFrame(get_keywords(PMID_list_3))

In [15]:
# 如果PMID有重复的，则保留第一个
df = df_list_3.drop_duplicates(subset=["PMID"], keep="first")

In [24]:
df

,PMID,DOI,Author,Title,Abstract,Keywords,Journal,Institution,Country,Date,Year
0,25025273,Crit Rev Biotechnol. 2016;36(1):59-69. doi: 10...,"[Singh, Bijender]",Myceliophthora thermophila syn. Sporotrichum t...,Myceliophthora thermophila syn. Sporotrichum t...,"[Biomolecules, Myceliophthora thermophila, Spo...",Critical reviews in biotechnology,"a Laboratory of Bioprocess Technology, Departm...",India,20140715,2016
1,32640074,J Appl Microbiol. 2021 Jan;130(1):90-99. doi: ...,"[Dos Santos Gomes, A C, Casciatori, F P, Gomes...",Growth kinetics of Myceliophthora thermophila ...,AIMS: This work aimed to estimate the growth o...,"[N-acetylglucosamine, enzymes, growth kinetics...",Journal of applied microbiology,"Instituto de Biociencias, Letras e Ciencias Ex...",Brazil,20200729,2021 Jan
4,31534479,Biotechnol Biofuels. 2019 Sep 16;12:220. doi: ...,"[Dos Santos Gomes, Ana Carolina, Falkoski, Dan...",Myceliophthora thermophila Xyr1 is predominant...,BACKGROUND: Myceliophthora thermophila is a th...,"[Cellulose degradation, Myceliophthora thermop...",Biotechnology for biofuels,"1Fungal Physiology, Westerdijk Fungal Biodiver...",The Netherlands. ISNI: 0000000120346234. GRID:...,20190916,2019
5,24995002,Front Microbiol. 2014 Jun 18;5:281. doi: 10.33...,"[Karnaouri, Anthi, Topakas, Evangelos, Antonop...",Genomic insights into the fungal lignocellulol...,The microbial conversion of solid cellulosic b...,"[CAZy, Myceliophthora thermophila, biofuels, l...",Frontiers in microbiology,"Biotechnology Laboratory, Department of Synthe...",Sweden,20140618,2014
6,35450635,Carbohydr Polym. 2022 Jul 15;288:119373. doi: ...,"[Sun, Peicheng, de Munnik, Melanie, van Berkel...",Extending the diversity of Myceliophthora ther...,Lytic polysaccharide monooxygenases (LPMOs) pl...,"[Active site segment, LPMOs, Lignocellulose, M...",Carbohydrate polymers,"Laboratory of Food Chemistry, Wageningen Unive...",the Netherlands. Electronic address: peicheng....,20220318,2022 Jul 15
...,...,...,...,...,...,...,...,...,...,...,...
275,38029111,Front Microbiol. 2023 Nov 3;14:1230738. doi: 1...,"[Wen, Jiaqi, Miao, Ting, Basit, Abdul, Li, Qun...",Highly efficient synergistic activity of an al...,"Here, an alpha-L-arabinofuranosidase (termed T...","[arabinoxylan, catalytic residues, enzyme syne...",Frontiers in microbiology,State Key Laboratory of Animal Biotech Breedin...,China,20231103,2023
276,33807631,Microorganisms. 2021 Mar 5;9(3):533. doi: 10.3...,"[Contato, Alex Graca, de Oliveira, Tassio Brit...",Prospection of Fungal Lignocellulolytic Enzyme...,The lignocellulosic biomass comprises three ma...,"[Hymenaea courbaril, Tamarindus indica, biopro...",Microorganisms,"Departamento de Bioquimica e Imunologia, Facul...",Brazil,20210305,2021 Mar 5
277,12817956,Inorg Chem. 2003 Jun 30;42(13):4006-17. doi: 1...,"[Palmer, Amy E, Szilagyi, Robert K, Cherry, Jo...",Spectroscopic characterization of the Leu513Hi...,"A variety of spectroscopic techniques, combine...",?,Inorganic chemistry,"Department of Chemistry, Stanford University, ...",USA,?,2003 Jun 30
278,30383933,Mol Gen Mikrobiol Virusol. 2016;34(3):109-114.,"[Krestyanova, I N, Sakhibgaraeva, L F, Berezin...",[CHARACTERISTICS OF FUNGAL STRAINS PRODUCING T...,Fungal strains degrading plant biomass availab...,?,"Molekuliarnaia genetika, mikrobiologiia i viru...",?,,?,2016


In [133]:
# # 将df中的DOI和Publication Year填充
# for i in range(len(df)):
#     for j in range(len(df_Mt_filter)):
#         if df.loc[i, "PMID"] == df_Mt_filter.loc[j, "PMID"]:
#             doi = df_Mt_filter.loc[j, "DOI"]
#             pub_year = df_Mt_filter.loc[j, "Publication Year"]
#             if pd.isnull(doi):
#                 doi = ""
#             if pd.isnull(pub_year):
#                 pub_year = ""
#             df.loc[i, "DOI"] = doi
#             df.loc[i, "Publication Year"] = pub_year

In [30]:
# 创建一个副本，免得操作失误
df_copy = df.copy()

In [31]:
# Keywords列中的值转换为str
df_copy["Keywords"] = df_copy["Keywords"].astype(str)

# 判断Keywords列的值是否为?
# for i in df_copy.index:
for i, row in df_copy.iterrows():

    if df_copy.loc[i, "Keywords"] == "?":
        # 将?替换为空
        df_copy.loc[i, "Keywords"] = ""
    else:
        # 去掉Keywords列中的[]
        df_copy.loc[i, "Keywords"] = df_copy.loc[i, "Keywords"].strip("[]")
        # 将单引号替换为空
        df_copy.loc[i, "Keywords"] = df_copy.loc[i, "Keywords"].replace("'", "")

df_copy

,PMID,DOI,Author,Title,Abstract,Keywords,Journal,Institution,Country,Date,Year
0,25025273,Crit Rev Biotechnol. 2016;36(1):59-69. doi: 10...,"[Singh, Bijender]",Myceliophthora thermophila syn. Sporotrichum t...,Myceliophthora thermophila syn. Sporotrichum t...,"Biomolecules, Myceliophthora thermophila, Spor...",Critical reviews in biotechnology,"a Laboratory of Bioprocess Technology, Departm...",India,20140715,2016
1,32640074,J Appl Microbiol. 2021 Jan;130(1):90-99. doi: ...,"[Dos Santos Gomes, A C, Casciatori, F P, Gomes...",Growth kinetics of Myceliophthora thermophila ...,AIMS: This work aimed to estimate the growth o...,"N-acetylglucosamine, enzymes, growth kinetics,...",Journal of applied microbiology,"Instituto de Biociencias, Letras e Ciencias Ex...",Brazil,20200729,2021 Jan
4,31534479,Biotechnol Biofuels. 2019 Sep 16;12:220. doi: ...,"[Dos Santos Gomes, Ana Carolina, Falkoski, Dan...",Myceliophthora thermophila Xyr1 is predominant...,BACKGROUND: Myceliophthora thermophila is a th...,"Cellulose degradation, Myceliophthora thermoph...",Biotechnology for biofuels,"1Fungal Physiology, Westerdijk Fungal Biodiver...",The Netherlands. ISNI: 0000000120346234. GRID:...,20190916,2019
5,24995002,Front Microbiol. 2014 Jun 18;5:281. doi: 10.33...,"[Karnaouri, Anthi, Topakas, Evangelos, Antonop...",Genomic insights into the fungal lignocellulol...,The microbial conversion of solid cellulosic b...,"CAZy, Myceliophthora thermophila, biofuels, li...",Frontiers in microbiology,"Biotechnology Laboratory, Department of Synthe...",Sweden,20140618,2014
6,35450635,Carbohydr Polym. 2022 Jul 15;288:119373. doi: ...,"[Sun, Peicheng, de Munnik, Melanie, van Berkel...",Extending the diversity of Myceliophthora ther...,Lytic polysaccharide monooxygenases (LPMOs) pl...,"Active site segment, LPMOs, Lignocellulose, Ma...",Carbohydrate polymers,"Laboratory of Food Chemistry, Wageningen Unive...",the Netherlands. Electronic address: peicheng....,20220318,2022 Jul 15
...,...,...,...,...,...,...,...,...,...,...,...
275,38029111,Front Microbiol. 2023 Nov 3;14:1230738. doi: 1...,"[Wen, Jiaqi, Miao, Ting, Basit, Abdul, Li, Qun...",Highly efficient synergistic activity of an al...,"Here, an alpha-L-arabinofuranosidase (termed T...","arabinoxylan, catalytic residues, enzyme syner...",Frontiers in microbiology,State Key Laboratory of Animal Biotech Breedin...,China,20231103,2023
276,33807631,Microorganisms. 2021 Mar 5;9(3):533. doi: 10.3...,"[Contato, Alex Graca, de Oliveira, Tassio Brit...",Prospection of Fungal Lignocellulolytic Enzyme...,The lignocellulosic biomass comprises three ma...,"Hymenaea courbaril, Tamarindus indica, biopros...",Microorganisms,"Departamento de Bioquimica e Imunologia, Facul...",Brazil,20210305,2021 Mar 5
277,12817956,Inorg Chem. 2003 Jun 30;42(13):4006-17. doi: 1...,"[Palmer, Amy E, Szilagyi, Robert K, Cherry, Jo...",Spectroscopic characterization of the Leu513Hi...,"A variety of spectroscopic techniques, combine...",,Inorganic chemistry,"Department of Chemistry, Stanford University, ...",USA,?,2003 Jun 30
278,30383933,Mol Gen Mikrobiol Virusol. 2016;34(3):109-114.,"[Krestyanova, I N, Sakhibgaraeva, L F, Berezin...",[CHARACTERISTICS OF FUNGAL STRAINS PRODUCING T...,Fungal strains degrading plant biomass availab...,,"Molekuliarnaia genetika, mikrobiologiia i viru...",?,,?,2016


In [32]:
# Yead列只保留前4位
df_copy["Year"] = df_copy["Year"].str[:4]
# df_copy按照Year列进行降序排列
df_copy = df_copy.sort_values(by="Year", ascending=False)
df_copy

,PMID,DOI,Author,Title,Abstract,Keywords,Journal,Institution,Country,Date,Year
110,37977536,Chemosphere. 2024 Jan;348:140733. doi: 10.1016...,"[Diefenbach, Thore, Sumetzberger-Hasinger, Mar...",Laccase-mediated degradation of petroleum hydr...,Laccases (EC1.10.3.2) have attracted growing a...,"Bioremediation, Laccase, Polyaromatic hydrocar...",Chemosphere,"Institute of Environmental Biotechnology, Univ...",Austria,20231115,2024
236,37354337,Biotechnol Lett. 2023 Sep;45(9):1093-1102. doi...,"[Contato, Alex Graca, Nogueira, Karoline Maria...",Trichoderma longibrachiatum and thermothelomyc...,OBJECTIVES: The aim of the present work was to...,"Co-culture, Saccharification, Sugarcane bagass...",Biotechnology letters,"Departamento de Bioquimica e Imunologia, Facul...",Brazil,20230624,2023
245,37869709,Front Bioeng Biotechnol. 2023 Oct 6;11:1279146...,"[Siebecker, Benedikt, Schutze, Tabea, Spohner,...",Transcriptomic insights into the roles of the ...,"Introduction: Thermothelomyces thermophilus, f...","Clr, Myceliophthora thermophila, Thermothelomy...",Frontiers in bioengineering and biotechnology,"Chair of Applied and Molecular Microbiology, I...",Germany,20231006,2023
150,36916669,Org Biomol Chem. 2023 Mar 29;21(13):2742-2747....,"[Shahedi, Mansour, Omidi, Niloofar, Habibi, Zo...",Biocatalytic stereoselective synthesis of pyrr...,"Highly functionalized pyrrolidine-2,3-diones c...",,Organic & biomolecular chemistry,"Department of Organic Chemistry, Shahid Behesh...",Iran. Z_habibi@sbu.ac.ir,20230329,2023
83,36966330,Biotechnol Biofuels Bioprod. 2023 Mar 25;16(1)...,"[Gu, Shuying, Zhao, Zhen, Xue, Fanglei, Liu, D...",The arabinose transporter MtLat-1 is involved ...,BACKGROUND: Filamentous fungi possess an array...,"Hemicellulase, L-Arabinose transport, MtAra-1,...",Biotechnology for biofuels and bioproducts,Key Laboratory of Engineering Biology for Low-...,China,20230325,2023
...,...,...,...,...,...,...,...,...,...,...,...
177,24425278,World J Microbiol Biotechnol. 1991 Nov;7(6):61...,"[Roy, S K, Raha, S K, Sadhukhan, R K, Chakraba...",Purification and characterization of extracell...,The extracellular beta-glucosidase has been pu...,,World journal of microbiology & biotechnology,", .",,?,1991
106,2269871,J Gen Microbiol. 1990 Oct;136(10):1967-71. doi...,"[Roy, S K, Dey, S K, Raha, S K, Chakrabarty, S L]",Purification and properties of an extracellula...,"An extracellular endoglucanase (1,4-beta-gluca...",,Journal of general microbiology,"Department of Microbiology, Bose Institute, Ca...",India,?,1990
251,2793591,J Antibiot (Tokyo). 1989 Sep;42(9):1362-9. doi...,"[Yaginuma, S, Asahi, A, Morishita, A, Hayashi,...",Isolation and characterization of new thiol pr...,"New thiol protease inhibitors, estatins A and ...",,The Journal of antibiotics,"Exploratory Research Laboratory, Toyo Jozo Co....",Japan,?,1989
111,16347725,Appl Environ Microbiol. 1988 Aug;54(8):2152-3....,"[Roy, S K, Raha, S K, Dey, S K, Chakrabarty, S L]",Induction and Catabolite Repression of beta-Gl...,beta-Glucosidase activity in Myceliophthora th...,,Applied and environmental microbiology,"Department of Microbiology, Bose Institute, 93...",India,?,1988


In [33]:
# 对df_copy重新排序
df_paper=df_copy[['PMID', 'Title', 'Author','Abstract', 'Keywords', 'Journal', 'Institution', 'Country', 'Year', 'DOI']].copy()

,PMID,Title,Author,Abstract,Keywords,Journal,Institution,Country,Year,DOI
110,37977536,Laccase-mediated degradation of petroleum hydr...,"[Diefenbach, Thore, Sumetzberger-Hasinger, Mar...",Laccases (EC1.10.3.2) have attracted growing a...,"Bioremediation, Laccase, Polyaromatic hydrocar...",Chemosphere,"Institute of Environmental Biotechnology, Univ...",Austria,2024,Chemosphere. 2024 Jan;348:140733. doi: 10.1016...
236,37354337,Trichoderma longibrachiatum and thermothelomyc...,"[Contato, Alex Graca, Nogueira, Karoline Maria...",OBJECTIVES: The aim of the present work was to...,"Co-culture, Saccharification, Sugarcane bagass...",Biotechnology letters,"Departamento de Bioquimica e Imunologia, Facul...",Brazil,2023,Biotechnol Lett. 2023 Sep;45(9):1093-1102. doi...
245,37869709,Transcriptomic insights into the roles of the ...,"[Siebecker, Benedikt, Schutze, Tabea, Spohner,...","Introduction: Thermothelomyces thermophilus, f...","Clr, Myceliophthora thermophila, Thermothelomy...",Frontiers in bioengineering and biotechnology,"Chair of Applied and Molecular Microbiology, I...",Germany,2023,Front Bioeng Biotechnol. 2023 Oct 6;11:1279146...
150,36916669,Biocatalytic stereoselective synthesis of pyrr...,"[Shahedi, Mansour, Omidi, Niloofar, Habibi, Zo...","Highly functionalized pyrrolidine-2,3-diones c...",,Organic & biomolecular chemistry,"Department of Organic Chemistry, Shahid Behesh...",Iran. Z_habibi@sbu.ac.ir,2023,Org Biomol Chem. 2023 Mar 29;21(13):2742-2747....
83,36966330,The arabinose transporter MtLat-1 is involved ...,"[Gu, Shuying, Zhao, Zhen, Xue, Fanglei, Liu, D...",BACKGROUND: Filamentous fungi possess an array...,"Hemicellulase, L-Arabinose transport, MtAra-1,...",Biotechnology for biofuels and bioproducts,Key Laboratory of Engineering Biology for Low-...,China,2023,Biotechnol Biofuels Bioprod. 2023 Mar 25;16(1)...
...,...,...,...,...,...,...,...,...,...,...
177,24425278,Purification and characterization of extracell...,"[Roy, S K, Raha, S K, Sadhukhan, R K, Chakraba...",The extracellular beta-glucosidase has been pu...,,World journal of microbiology & biotechnology,", .",,1991,World J Microbiol Biotechnol. 1991 Nov;7(6):61...
106,2269871,Purification and properties of an extracellula...,"[Roy, S K, Dey, S K, Raha, S K, Chakrabarty, S L]","An extracellular endoglucanase (1,4-beta-gluca...",,Journal of general microbiology,"Department of Microbiology, Bose Institute, Ca...",India,1990,J Gen Microbiol. 1990 Oct;136(10):1967-71. doi...
251,2793591,Isolation and characterization of new thiol pr...,"[Yaginuma, S, Asahi, A, Morishita, A, Hayashi,...","New thiol protease inhibitors, estatins A and ...",,The Journal of antibiotics,"Exploratory Research Laboratory, Toyo Jozo Co....",Japan,1989,J Antibiot (Tokyo). 1989 Sep;42(9):1362-9. doi...
111,16347725,Induction and Catabolite Repression of beta-Gl...,"[Roy, S K, Raha, S K, Dey, S K, Chakrabarty, S L]",beta-Glucosidase activity in Myceliophthora th...,,Applied and environmental microbiology,"Department of Microbiology, Bose Institute, 93...",India,1988,Appl Environ Microbiol. 1988 Aug;54(8):2152-3....


In [42]:
import re

def extract_dois_with_na(doi_list):
    """
    从给定的DOI列中提取DOI号。如果某个条目中没有找到DOI号，则在结果列表中用'NA'填充该位置。

    参数:
        doi_list (list): 包含DOI信息的字符串列表。

    返回:
        list: 包含DOI号或'NA'的列表。
    """
    # 定义用于匹配DOI号的正则表达式
    doi_pattern = r'doi:\s*([\d\.\/\w-]+)'

    # 初始化一个空列表，用于存储提取的DOI号或'NA'
    extracted_dois = []

    # 遍历列表中的每个字符串
    for doi in doi_list:
        # 使用正则表达式查找DOI号
        match = re.search(doi_pattern, doi)
        if match:
            # 如果找到了匹配项，则将DOI号添加到结果列表中
            extracted_dois.append(match.group(1))
        else:
            # 如果没有找到匹配项，则在该位置添加'NA'
            extracted_dois.append('NA')

    return extracted_dois

# 示例：应用这个函数到一个测试数据列表，包括一个没有DOI的条目
test_data_with_na = [
    "Chemosphere. 2024 Jan;348:140733. doi: 10.1016/j.chemosphere.2023.140733. Epub 2023 Nov 15.",
    "Biotechnol Lett. 2023 Sep;45(9):1093-1102. doi: 10.1007/s10529-023-03395-7. Epub 2023 Jun 24.",
    "一个没有DOI的条目"
]

# 调用函数并输出结果
df_paper["DOI"] = extract_dois_with_na(df_paper["DOI"])

In [43]:
df_paper

,PMID,Title,Author,Abstract,Keywords,Journal,Institution,Country,Year,DOI
110,37977536,Laccase-mediated degradation of petroleum hydr...,"[Diefenbach, Thore, Sumetzberger-Hasinger, Mar...",Laccases (EC1.10.3.2) have attracted growing a...,"Bioremediation, Laccase, Polyaromatic hydrocar...",Chemosphere,"Institute of Environmental Biotechnology, Univ...",Austria,2024,10.1016/j.chemosphere.2023.140733.
236,37354337,Trichoderma longibrachiatum and thermothelomyc...,"[Contato, Alex Graca, Nogueira, Karoline Maria...",OBJECTIVES: The aim of the present work was to...,"Co-culture, Saccharification, Sugarcane bagass...",Biotechnology letters,"Departamento de Bioquimica e Imunologia, Facul...",Brazil,2023,10.1007/s10529-023-03395-7.
245,37869709,Transcriptomic insights into the roles of the ...,"[Siebecker, Benedikt, Schutze, Tabea, Spohner,...","Introduction: Thermothelomyces thermophilus, f...","Clr, Myceliophthora thermophila, Thermothelomy...",Frontiers in bioengineering and biotechnology,"Chair of Applied and Molecular Microbiology, I...",Germany,2023,10.3389/fbioe.2023.1279146.
150,36916669,Biocatalytic stereoselective synthesis of pyrr...,"[Shahedi, Mansour, Omidi, Niloofar, Habibi, Zo...","Highly functionalized pyrrolidine-2,3-diones c...",,Organic & biomolecular chemistry,"Department of Organic Chemistry, Shahid Behesh...",Iran. Z_habibi@sbu.ac.ir,2023,10.1039/d2ob02294j.
83,36966330,The arabinose transporter MtLat-1 is involved ...,"[Gu, Shuying, Zhao, Zhen, Xue, Fanglei, Liu, D...",BACKGROUND: Filamentous fungi possess an array...,"Hemicellulase, L-Arabinose transport, MtAra-1,...",Biotechnology for biofuels and bioproducts,Key Laboratory of Engineering Biology for Low-...,China,2023,10.1186/s13068-023-02305-3.
...,...,...,...,...,...,...,...,...,...,...
177,24425278,Purification and characterization of extracell...,"[Roy, S K, Raha, S K, Sadhukhan, R K, Chakraba...",The extracellular beta-glucosidase has been pu...,,World journal of microbiology & biotechnology,", .",,1991,10.1007/BF00452843.
106,2269871,Purification and properties of an extracellula...,"[Roy, S K, Dey, S K, Raha, S K, Chakrabarty, S L]","An extracellular endoglucanase (1,4-beta-gluca...",,Journal of general microbiology,"Department of Microbiology, Bose Institute, Ca...",India,1990,10.1099/00221287-136-10-1967.
251,2793591,Isolation and characterization of new thiol pr...,"[Yaginuma, S, Asahi, A, Morishita, A, Hayashi,...","New thiol protease inhibitors, estatins A and ...",,The Journal of antibiotics,"Exploratory Research Laboratory, Toyo Jozo Co....",Japan,1989,10.7164/antibiotics.42.1362.
111,16347725,Induction and Catabolite Repression of beta-Gl...,"[Roy, S K, Raha, S K, Dey, S K, Chakrabarty, S L]",beta-Glucosidase activity in Myceliophthora th...,,Applied and environmental microbiology,"Department of Microbiology, Bose Institute, 93...",India,1988,10.1128/aem.54.8.2152-2153.1988.


In [44]:
# df_copy.to_excel("Mt_paper_all.xlsx", index=False)

# 保存为csv
df_paper.to_csv("mt/Mt_paper_20231211.csv", index=False)